In [ ]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2021-05-11 09:47:45--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip.2’

source.zip.2        100%[===================>]   2.64K  --.-KB/s    in 0s      

2021-05-11 09:47:46 (544 MB/s) - ‘source.zip.2’ saved [2707/2707]



In [ ]:
!unzip source.zip

Archive:  source.zip
replace source3.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace source3.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


In [ ]:
tmpfile = "temp.tmp"
logfile = "logfile.txt"
targetfile = "transformed_data.csv"

Define function to extract CSV

In [ ]:
def extract_from_csv(file_to_process):
  dataframe = pd.read_csv(file_to_process)
  return dataframe

Define function to extract JSON

In [ ]:
def extract_from_json(file_to_process):
  dataframe = pd.read_json(file_to_process,lines=True)
  return dataframe

Define function to extract XML

In [ ]:
def extract_from_xml(file_to_process):
  dataframe = pd.DataFrame(columns=["name","height","weight"])
  tree = ET.parse(file_to_process)
  root = tree.getroot()
  for person in root:
    name = person.find("name").text
    height = float(person.find("height").text)
    weight = float(person.find("weight").text)
    dataframe = dataframe.append({"name":name, "height":height, "weight":weight},ignore_index=True)
  return dataframe

In [ ]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

Transform data

In [ ]:
def transform(data):
  data['height'] = round(data.height * 0.0254,2)

  data['weight'] = round(data.weight * 0.45359237,2)

  return data

Loading

In [ ]:
def load(targetfile,data_to_load):
  data_to_load.to_csv(targetfile)

Logging

In [ ]:
def log(message):
  timestamp_format = '%Y-%h-%d-%H:%M:%S'
  now = datetime.now()
  timestamp = now.strftime(timestamp_format)
  with open("logfile.txt", "a") as f:
    f.write(timestamp + ',' + message + '\n')

Running ETL Process

In [ ]:
log("ETL Job Started")

In [ ]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,alex,65.78,112.99
6,ajay,71.52,136.49
7,alice,69.40,153.03
8,ravi,68.22,142.34
9,joe,67.79,144.30


In [ ]:
log("Transform phase Started")
transform_data = transform(extracted_data)
log("Transform phase Ended")
transform_data

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
5,alex,1.67,51.25
6,ajay,1.82,61.91
7,alice,1.76,69.41
8,ravi,1.73,64.56
9,joe,1.72,65.45


In [ ]:
log("Load phase Started")
load(targetfile,transform_data)
log("Load phase Ended")

In [ ]:
log("ETL job Ended")

# New Section

# New Section